**Импорт библиотек**

In [1]:
import requests
import pandas as pd

**Объявление переменных**

In [2]:
# Требуемые вакансии для поискового запроса
vacancies = 'data инженер, data engineer'

# Текст поискового запроса
search_text = ' and '.join(['\''+ elem + '\'' for elem in vacancies.split(', ')])

# url для парсинга
url = 'https://api.hh.ru/vacancies'

**Парсинг данных с сайта hh.ru**

*Подготовка датасета*

In [4]:
vacancies_data = []

# Цикл, проходящий по всему списку страниц поисковой выдачи и собирающий информацию о вакансиях

for i in range(requests.get(url, params={'text': search_text, 'per_page': 10, 'area':'113'}).json()['pages']):
    par = {'text': search_text, 'area':'113','per_page':'10', 'page':i}
    try:
        temp_json = requests.get(url, params=par).json()
        for elem in temp_json['items']:        
            try:
                temp_json_2 = requests.get(url + '/' + str(elem['id'])).json()
                vacancies_data.append([temp_json_2['name'],                                                                                    # название вакансии
                                    temp_json_2['experience']['name'],                                                                         # требуемые опыт
                                    temp_json_2['key_skills'],                                                                                 # требуемые навыки
                                    temp_json_2['employer']['name'],                                                                           # название компании-нанимателя
                                    requests.get('https://api.hh.ru/employers/' + str(temp_json_2['employer']['id'])).json()["industries"]])   # сфера деятельности компании-нанимателя
            
            except:
                pass
    except:
            pass
        
df = pd.DataFrame(columns=['vacancy_name', 'experience', 'skills', 'company', 'company_area'], data=vacancies_data)


In [6]:
df

,vacancy_name,experience,skills,company,company_area
0,Senior Data Scientist / ML Engineer,От 1 года до 3 лет,"[{'name': 'Docker'}, {'name': 'Python'}, {'nam...",Mindbox,"[{'id': '7.540', 'name': 'Разработка программн..."
1,Data scientist,От 3 до 6 лет,"[{'name': 'Python'}, {'name': 'PyTorch'}, {'na...",Астор,"[{'id': '7.539', 'name': 'Системная интеграция..."
2,Data Engineer,Нет опыта,"[{'name': 'SQL'}, {'name': 'Python'}, {'name':...",Апптимизм,"[{'id': '7.541', 'name': 'Интернет-компания (п..."
3,Data Engineer,От 3 до 6 лет,"[{'name': 'Английский язык'}, {'name': 'SQL'},...","Hi, Rockits!","[{'id': '39.441', 'name': 'Научно-исследовател..."
4,Data engineer Инженер данных (Аналитик),От 1 года до 3 лет,"[{'name': 'Python'}, {'name': 'Hadoop'}, {'nam...",Императив Медиа,[]
...,...,...,...,...,...
1134,Архитектор,От 1 года до 3 лет,[],МТС,"[{'id': '9.399', 'name': 'Мобильная связь'}]"
1135,QA Automation Engineer (релокация на Кипр),От 3 до 6 лет,[],Soveren,"[{'id': '7.540', 'name': 'Разработка программн..."
1136,Аналитик по моделированию данных,От 1 года до 3 лет,"[{'name': 'UML'}, {'name': 'Системный анализ'}...",Московский метрополитен,"[{'id': '5.463', 'name': 'Железнодорожные пере..."
1137,Trading Platform Support Engineer,От 3 до 6 лет,[],Носов Константин Константинович,[]


*Обработка датасета*

In [7]:
# Преобразование данных о требуемых навыках и сферах деятельности компании в читаемый вид
df['skills'] = df['skills'].apply(lambda x: [y['name'] for y in x])
df['company_area'] = df['company_area'].apply(lambda x: [y['name'] for y in x])

# Создание столбца с собственными индексами вакансий для сохранения их уникальности
df['vacancy_index'] = df.index

# Развертка списков навыков и сфер работы компаний для дальнейшего анализа
df = df.explode('company_area').explode('skills').reset_index(drop=True)

*Анализ данных*

In [8]:
print('Число обнаруженных вакансий: {}'.format(len(df.groupby('vacancy_index'))))

Число обнаруженных вакансий: 1139


In [9]:
print('Число компаний, разместивших вакансии: {}'.format(len(df.groupby('company'))))

Число компаний, разместивших вакансии: 405


In [10]:
print('Распределение требований к опыту по числу вакансий:')
for i, k in df.groupby('experience')['vacancy_index'].nunique().sort_values(ascending=False).head(10).items():
    print('Требуемый опыт ' + i + ' - ' + str(k) + ' вакансий')

Распределение требований к опыту по числу вакансий:
Требуемый опыт От 3 до 6 лет - 572 вакансий
Требуемый опыт От 1 года до 3 лет - 433 вакансий
Требуемый опыт Более 6 лет - 90 вакансий
Требуемый опыт Нет опыта - 44 вакансий


In [11]:
print('Топ-10 отраслей компаний, которым требуются инженеры данных:')
for i, k in df.groupby('company_area')['vacancy_index'].nunique().sort_values(ascending=False).head(10).items():
    print(i + ' - ' + str(k) + ' вакансий')

Топ-10 отраслей компаний, которым требуются инженеры данных:
Разработка программного обеспечения - 342 вакансий
Системная интеграция,  автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг - 335 вакансий
Банк - 212 вакансий
Интернет-компания (поисковики, платежные системы, соц.сети, информационно-познавательные и развлекательные ресурсы, продвижение сайтов и прочее) - 211 вакансий
Мобильная связь - 85 вакансий
Коллекторская деятельность - 54 вакансий
Интернет-провайдер - 42 вакансий
Фиксированная связь - 39 вакансий
Интернет-магазин - 38 вакансий
Кадровые агентства - 33 вакансий


In [12]:
print('Список требуемых навыков-технологий (языки программирования, фреймворки): ')
for i, k in df.groupby('skills')['vacancy_index'].nunique().sort_values(ascending=False).items():
    print(i + ' - ' + str(k) + ' вакансий')

Список требуемых навыков-технологий (языки программирования, фреймворки): 
Python - 467 вакансий
SQL - 425 вакансий
Linux - 192 вакансий
Английский язык - 183 вакансий
PostgreSQL - 172 вакансий
Hadoop - 155 вакансий
Java - 155 вакансий
Git - 144 вакансий
Spark - 118 вакансий
ETL - 105 вакансий
Docker - 89 вакансий
Kafka - 87 вакансий
Big Data - 82 вакансий
SCALA - 67 вакансий
Kubernetes - 54 вакансий
DWH - 53 вакансий
Data Analysis - 48 вакансий
English - 46 вакансий
Базы данных - 46 вакансий
Анализ данных - 46 вакансий
Bash - 40 вакансий
Machine Learning - 40 вакансий
ORACLE - 39 вакансий
Ansible - 38 вакансий
MS SQL - 38 вакансий
Atlassian Jira - 38 вакансий
Spring Framework - 37 вакансий
AWS - 37 вакансий
C/C++ - 36 вакансий
ClickHouse - 36 вакансий
Работа с базами данных - 32 вакансий
Hive - 31 вакансий
Airflow - 28 вакансий
Pandas - 28 вакансий
CI/CD - 27 вакансий
C++ - 27 вакансий
Управление проектами - 26 вакансий
Golang - 26 вакансий
Fluent English - 25 вакансий
JavaScript - 25

In [13]:
print('ТОП-10 требуемых навыков-технологий (языки программирования, фреймворки): ')
for i, k in df.groupby('skills')['vacancy_index'].nunique().sort_values(ascending=False).head(10).items():
    print(i + ' - ' + str(k) + ' вакансий')

ТОП-10 требуемых навыков-технологий (языки программирования, фреймворки): 
Python - 467 вакансий
SQL - 425 вакансий
Linux - 192 вакансий
Английский язык - 183 вакансий
PostgreSQL - 172 вакансий
Hadoop - 155 вакансий
Java - 155 вакансий
Git - 144 вакансий
Spark - 118 вакансий
ETL - 105 вакансий


**Отчет об исследовании**

Для анализа рынка труда по профессии данных был выбран сайт поиска работы hh.ru. С помощью библиотек request и pandas был осуществлен сбор данных и оформлен датасет, содержащий в себе вакансии, найденные в поиске по ключевым словам "инженер данных" и "data engineer". 

По результатам анализа можем сделать следующие выводы:
1. Было обнаружено 1139 вакансий по требуемым ключевым словам. Эти вакансии были размещены 405 компаниями разного размера. В основном это компании, занимающиеся разработкой цифровых решений,IT-консалтингом, а также предприятия банковской сферы.
2. В основном требуются специалисты опытом от 1 до 6 лет, владеющие языками Python и Java, знающие ETL, Hadoop, Spark, SQL
3. Python и Java - языки программирования, используются для разработки; SQL - язык запросов, применяемый при управлении базами данных; Hadoop и Spark фремворки, применяемые для распределенной обработки данных.
4. Цель обучения: получение новых знаний и навыков работы с данными с точки зрения инженерного подхода. Также я хочу научиться пользоваться такими инструментами как Hadoop, Kafka, Spark, Airflow.